<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/-/blob/main/%E3%80%90note%E5%85%AC%E9%96%8B%E3%80%91_%E5%B0%8F%E5%AD%A6%E7%94%9F%E5%90%91%E3%81%91%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E3%82%B9%E3%82%AF%E3%83%BC%E3%83%AB_AI%E3%82%AB%E3%83%AA%E3%82%AD%E3%83%A5%E3%83%A9%E3%83%A0%E4%BD%9C%E6%88%90%E3%83%84%E3%83%BC%E3%83%AB20250527.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# 🚀 Google Colab 高速処理版 - プログラミングスクールAI
# 処理速度を大幅改善した最適化版

print("⚡ Google Colab高速処理版 プログラミングスクールAI")
print("=" * 50)

# ===== 1. 高速インストール =====
import subprocess
import sys

def fast_install():
    """高速インストール"""
    print("⚡ 高速インストール中...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "openai", "ipywidgets", "-q"], timeout=30)
        print("✅ インストール完了")
    except:
        print("⚠️ インストールをスキップ")

fast_install()

# ===== 2. 必要なインポートのみ =====
try:
    from openai import OpenAI
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    import os
    print("✅ ライブラリ読み込み完了")
except ImportError as e:
    print(f"❌ インポートエラー: {e}")

# ===== 3. 環境設定 =====
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

client = None

def quick_setup():
    """高速セットアップ"""
    global client
    try:
        if IN_COLAB:
            api_key = userdata.get('OPENAI_API_KEY')
        else:
            api_key = os.getenv('OPENAI_API_KEY')

        client = OpenAI(api_key=api_key)

        # 高速接続テスト
        client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": "Hi"}],
            max_tokens=1,
            timeout=5
        )
        return True
    except:
        return False

# ===== 4. 高速AIクラス =====
class FastProgrammingAI:
    """高速処理版AI"""

    def __init__(self):
        self.output = widgets.Output()
        self.settings = {}

        # 最適化されたデータ
        self.grades = ["低学年", "中学年", "高学年"]
        self.topics = ["Scratch", "ゲーム", "ロボット", "Web", "データ"]
        self.functions = {
            "📚 カリキュラム": self.curriculum,
            "📖 レッスン": self.lesson,
            "❓ クイズ": self.quiz,
            "🚀 プロジェクト": self.project
        }

    def btn(self, text, callback, style='primary'):
        """ボタン作成"""
        b = widgets.Button(
            description=text,
            button_style=style,
            layout=widgets.Layout(width='250px', height='35px')
        )
        b.on_click(callback)
        return b

    def start(self):
        """開始画面"""
        with self.output:
            clear_output()
            print("⚡ 高速プログラミング教材AI")

        display(widgets.VBox([
            widgets.HTML("<h3>⚡ 高速AI教材生成</h3>"),
            self.btn("🚀 開始", self.menu, 'success')
        ]))
        display(self.output)

    def menu(self, b=None):
        """メニュー"""
        with self.output:
            clear_output()
            print("📋 機能選択")

        buttons = [self.btn(name, lambda x, f=func: self.select_func(f))
                  for name, func in self.functions.items()]
        buttons.append(self.btn("🏠 戻る", self.start, 'warning'))

        display(widgets.VBox(buttons))

    def select_func(self, func):
        """機能選択"""
        self.current_func = func
        self.grade_menu()

    def grade_menu(self):
        """学年選択"""
        with self.output:
            clear_output()
            print("📚 学年選択")

        buttons = [self.btn(f"小学{grade}", lambda x, g=grade: self.select_grade(g))
                  for grade in self.grades]
        buttons.append(self.btn("⬅️ 戻る", self.menu, 'warning'))

        display(widgets.VBox(buttons))

    def select_grade(self, grade):
        """学年選択処理"""
        self.settings['grade'] = grade
        self.topic_menu()

    def topic_menu(self):
        """トピック選択"""
        with self.output:
            clear_output()
            print("📝 テーマ選択")

        buttons = [self.btn(topic, lambda x, t=topic: self.select_topic(t))
                  for topic in self.topics]
        buttons.append(self.btn("⬅️ 戻る", self.grade_menu, 'warning'))

        display(widgets.VBox(buttons))

    def select_topic(self, topic):
        """トピック選択処理"""
        self.settings['topic'] = topic
        self.execute()

    def execute(self):
        """AI実行"""
        with self.output:
            clear_output()
            print("⚡ AI高速処理中...")
            print(f"学年: {self.settings['grade']}")
            print(f"テーマ: {self.settings['topic']}")

        try:
            result = self.current_func()
            with self.output:
                clear_output()
                print("🎉 完成！")
                print("=" * 40)
                print(result)
                print("=" * 40)
        except Exception as e:
            with self.output:
                clear_output()
                print(f"❌ エラー: {e}")
                print("💡 APIキーを確認してください")

        display(self.btn("📋 メニュー", self.menu, 'primary'))

    # ===== 高速AI機能 =====
    def fast_ai(self, prompt, max_tokens=500):
        """高速AI呼び出し"""
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,  # 高速化のため低く設定
            max_tokens=max_tokens,
            timeout=15  # 15秒タイムアウト
        )
        return response.choices[0].message.content

    def curriculum(self):
        """カリキュラム生成"""
        prompt = f"""小学{self.settings['grade']}向け{self.settings['topic']}の4週間カリキュラムを簡潔に作成:
1. 学習目標3つ
2. 週次内容
3. 評価方法"""
        return self.fast_ai(prompt, 400)

    def lesson(self):
        """レッスンプラン"""
        prompt = f"""小学{self.settings['grade']}向け{self.settings['topic']}の60分レッスンプラン:
1. 導入(10分)
2. 展開(40分)
3. まとめ(10分)
各段階の活動内容を簡潔に"""
        return self.fast_ai(prompt, 400)

    def quiz(self):
        """クイズ生成"""
        prompt = f"""小学{self.settings['grade']}向け{self.settings['topic']}の4択クイズ3問:
各問題: 問題文、選択肢A-D、正解、解説"""
        return self.fast_ai(prompt, 600)

    def project(self):
        """プロジェクト提案"""
        prompt = f"""小学{self.settings['grade']}向け{self.settings['topic']}の楽しいプロジェクト2つ:
各プロジェクト: 名前、概要、時間、手順"""
        return self.fast_ai(prompt, 500)

# ===== 5. 高速起動関数 =====
def fast_start():
    """⚡ 高速起動"""
    print("⚡ 高速起動中...")

    if quick_setup():
        print("✅ セットアップ完了")
        app = FastProgrammingAI()
        app.start()
    else:
        print("❌ APIキー設定エラー")
        print("\n🔑 設定方法:")
        print("1. 左サイドバー🔑をクリック")
        print("2. New secret")
        print("3. Name: OPENAI_API_KEY")
        print("4. Value: あなたのAPIキー")
        print("5. Notebook access ON")

def speed_test():
    """速度テスト"""
    print("⚡ 速度テスト実行中...")
    if quick_setup():
        import time
        start = time.time()
        try:
            client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": "Hello"}],
                max_tokens=5,
                timeout=10
            )
            end = time.time()
            print(f"✅ レスポンス時間: {end-start:.2f}秒")
        except Exception as e:
            print(f"❌ テストエラー: {e}")
    else:
        print("❌ APIキー未設定")

# ===== 6. 使用方法 =====
print("\n⚡ 高速版使用方法")
print("=" * 25)
print("1. speed_test()   # 速度テスト")
print("2. fast_start()   # 高速起動")
print("\n🚀 高速化のポイント:")
print("- プロンプト最適化")
print("- トークン数削減")
print("- タイムアウト短縮")
print("- UI簡略化")
print("=" * 25)

⚡ Google Colab高速処理版 プログラミングスクールAI
⚡ 高速インストール中...
✅ インストール完了
✅ ライブラリ読み込み完了

⚡ 高速版使用方法
1. speed_test()   # 速度テスト
2. fast_start()   # 高速起動

🚀 高速化のポイント:
- プロンプト最適化
- トークン数削減
- タイムアウト短縮
- UI簡略化


In [18]:

#system_check()     # システム診断
start_ai()         # AI起動（改良版）
#emergency_stop()   # 緊急停止方法

🚀 プログラミングスクールAI - 安全起動版

🔐 OpenAI APIキーの設定
------------------------------
✅ Google Colab SecretsからAPIキーを取得
🔄 API接続テスト中... (最大10秒)
✅ API接続テスト成功
✅ 準備完了！

🎯 使い方:
下のボタンをクリックして操作してください
タイムアウト: 30秒で自動停止します


Output()

HTML(value='🔄 AI処理中... しばらくお待ちください')

Button(button_style='primary', description='📋 メニューに戻る', layout=Layout(height='40px', margin='5px', width='300p…

HTML(value='🔄 AI処理中... しばらくお待ちください')

Button(button_style='primary', description='📋 メニューに戻る', layout=Layout(height='40px', margin='5px', width='300p…